In [1]:
import os
import math
from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

# import import_ipynb
# from utils import to_timestamp, df_data_sensors, to_be_checked, is_corr_ok, df_to_csv, df_from_csv

# with the new api
from importnb import imports
with imports("ipynb"):
    from utils import to_timestamp, df_data_sensors, to_be_checked, is_corr_ok, df_to_csv, df_from_csv

PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## Temperature Raw Data

In [2]:
# Load Tangaras
df_tangaras = df_from_csv("tangaras.csv", dtindex=False)

print(f"Total Tangara Sensors: {len(df_tangaras)}")

df_tangaras.head()

Total Tangara Sensors: 9


ID  GEOHASH              MAC  \
0  TANGARA_260A  d29edyj  D29ESP32DE1260A   
1  TANGARA_4B1A  d29esj8  D29ESP32DE94B1A   
2  TANGARA_14D6  d29eg4k  D29ESP32DED14D6   
3  TANGARA_2B42  d29e6pg  D29ESP32DED2B42   
4  TANGARA_2E9A  d29ed62  D29ESP32DED2E9A   

                             GEOLOCATION  LATITUDE  LONGITUDE  
0  3.4613800048828125 -76.51222229003906  3.461380 -76.512222  
1  3.4586334228515625 -76.46415710449219  3.458633 -76.464157  
2  3.4847259521484375 -76.50260925292969  3.484726 -76.502609  
3  3.4270477294921875 -76.54792785644531  3.427048 -76.547928  
4  3.4407806396484375 -76.54106140136719  3.440781 -76.541061

In [3]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
START_ISO8601_DATETIME = os.getenv("START_ISO8601_DATETIME", None)
start_timestamp = to_timestamp(START_ISO8601_DATETIME)
# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
END_ISO8601_DATETIME = os.getenv("END_ISO8601_DATETIME", None)
end_timestamp = to_timestamp(os.getenv("END_ISO8601_DATETIME", None))

# GROUP BY TIME
GROUP_BY_TIME = os.getenv("GROUP_BY_TIME", None)

print(f'Since: {START_ISO8601_DATETIME} -> {start_timestamp}, Until: {END_ISO8601_DATETIME} -> {end_timestamp}')
print(f"Group by Time: {GROUP_BY_TIME}")

Since: 2023-11-06T23:00:00-05:00 -> 1699329600000, Until: 2023-11-07T00:00:00-05:00 -> 1699333200000
Group by Time: 30s


In [4]:
# Temperature Data Frame Sensors
df_temp_raw = df_data_sensors(df_tangaras, start_timestamp, end_timestamp, 'tmp', GROUP_BY_TIME)
df_temp_raw.head()
#df_temp_raw.shape

TANGARA_260A  TANGARA_4B1A  TANGARA_14D6  \
DATETIME                                                              
2023-11-06 23:00:00-05:00         26.84         25.61         27.34   
2023-11-06 23:00:30-05:00         26.82         25.58         27.34   
2023-11-06 23:01:00-05:00         26.79         25.56         27.34   
2023-11-06 23:01:30-05:00         26.78         25.55         27.31   
2023-11-06 23:02:00-05:00         26.82         25.55         27.29   

                           TANGARA_2B42  TANGARA_2E9A  TANGARA_2FF6  \
DATETIME                                                              
2023-11-06 23:00:00-05:00         26.45         29.16         26.44   
2023-11-06 23:00:30-05:00         26.44         29.16         26.42   
2023-11-06 23:01:00-05:00         26.44         29.19         26.43   
2023-11-06 23:01:30-05:00         26.45         29.20         26.44   
2023-11-06 23:02:00-05:00         26.43         29.23         26.43   

                           TANGARA_48C6  TANGARA_F1AE  TANGARA_06BE  
DATETIME                                                             
2023-11-06 23:00:00-05:00          26.4         28.09         27.41  
2023-11-06 23:00:30-05:00          26.4         28.08         27.41  
2023-11-06 23:01:00-05:00          26.4         28.12         27.42  
2023-11-06 23:01:30-05:00          26.4         28.09           NaN  
2023-11-06 23:02:00-05:00          26.4         28.06         27.45

## Descriptive Statistics

In [5]:
# Describe Data
df_temp_raw.describe()

TANGARA_260A  TANGARA_4B1A  TANGARA_14D6  TANGARA_2B42  TANGARA_2E9A  \
count    118.000000    114.000000    118.000000    116.000000    118.000000   
mean      27.097881     25.849298     27.262288     26.333707     29.144237   
std        0.197115      0.207111      0.061219      0.066640      0.089590   
min       26.780000     25.500000     27.150000     26.230000     28.990000   
25%       26.912500     25.660000     27.210000     26.270000     29.080000   
50%       27.100000     25.870000     27.260000     26.310000     29.120000   
75%       27.287500     26.025000     27.310000     26.400000     29.215000   
max       27.410000     26.140000     27.370000     26.450000     29.360000   

       TANGARA_2FF6  TANGARA_48C6  TANGARA_F1AE  TANGARA_06BE  
count    116.000000    114.000000    117.000000    114.000000  
mean      26.414310     26.261404     28.078547     27.424474  
std        0.052074      0.097317      0.022563      0.060149  
min       26.330000     26.100000     28.030000     27.310000  
25%       26.360000     26.200000     28.060000     27.380000  
50%       26.420000     26.300000     28.080000     27.420000  
75%       26.460000     26.300000     28.090000     27.460000  
max       26.520000     26.400000     28.130000     27.550000

## Missing Data

In [6]:
# Missing Data Threshold
threshold = 90
print(f'Threshold: {threshold}%')

# For each Tangara sensor
for id_tangara_sensor in df_temp_raw.columns:
    df_temp_sensor = df_temp_raw[[id_tangara_sensor]]
    # To be checked
    is_ok, data_percent, missing_data_percent = to_be_checked(df_temp_sensor, threshold)
    if not is_ok:
        print(f"Tangara Sensor: {id_tangara_sensor}, Data: {data_percent}%, Missing: {missing_data_percent}%, To be checked")

Threshold: 90%


/home/sebaxtian/Workspaces/Tangara/tangara-pipeline/standalone/notebooks/utils.ipynb:197: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  missing_data_percent = round(df_sensor.isna().sum()[0] * 100 / total)
/home/sebaxtian/Workspaces/Tangara/tangara-pipeline/standalone/notebooks/utils.ipynb:198: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data_percent = round(df_sensor.count()[0] * 100 / total)
/home/sebaxtian/Workspaces/Tangara/tangara-pipeline/standalone/notebooks/utils.ipynb:197: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

## Data Correlation

In [7]:
# Data Correlation Threshold
threshold = 0.9
print(f'Threshold: {threshold}%')

ID_REFE_TANGARA_SENSOR='TANGARA_06BE'
ID_TARG_TANGARA_SENSOR='TANGARA_2FF6'
# Reference Tangara Sensor
df_reference_sensor = df_temp_raw[ID_REFE_TANGARA_SENSOR]
# Target Tangara Sensor
df_target_sensor = df_temp_raw[ID_TARG_TANGARA_SENSOR]

# To be checked
if not math.isnan(df_reference_sensor.std()):
    is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
    if not is_ok:
        print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {ID_TARG_TANGARA_SENSOR}, Correlation: {corr}, To be checked")

# For each Tangara sensor
# for id_tangara_sensor in df_temp_raw.columns:
#     # Target Tangara Sensor
#     df_target_sensor = df_temp_raw[id_tangara_sensor]
#     # To be checked
#     is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
#     if not is_ok:
#         print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {id_tangara_sensor}, Correlation: {corr}, To be checked")


Threshold: 0.9%
Reference Tangara Sensor: TANGARA_06BE, Target Tangara Sensor: TANGARA_2FF6, Correlation: 0.0, To be checked


In [8]:
# Save Temperature Data Frame Sensors into CSV file
df_to_csv(df_temp_raw, "temp_raw.csv")